- https://www.kaggle.com/competitions/spaceship-titanic/data
- https://www.kaggle.com/code/dansbecker/your-first-machine-learning-model

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [ ]:
TRAIN_DATA: str = "data/train.csv"
TEST_DATA: str = "data/test.csv"

SEED: int = 2912

In [ ]:
train_df = pd.read_csv(TRAIN_DATA)
test_df = pd.read_csv(TEST_DATA)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.shape

In [ ]:
test_df.shape

In [ ]:
train_df.dtypes

In [ ]:
train_df["PassengerId"].nunique() == train_df.shape[0]

In [ ]:
test_df["PassengerId"].nunique() == test_df.shape[0]

In [ ]:
train_df["HomePlanet"].value_counts(dropna=False)

In [ ]:
train_df["CryoSleep"].value_counts(dropna=False)

In [ ]:
train_df["Destination"].value_counts(dropna=False)

In [ ]:
train_df["VIP"].value_counts(dropna=False)

In [ ]:
# Target
train_df["Transported"].value_counts(dropna=False)

In [ ]:
train_df["Name"].nunique()

In [ ]:
for col in train_df:
    print(col)
    display(train_df[col].unique())
    print("---")

In [ ]:
train_df.isna().sum()

In [ ]:
test_df.isna().sum()

In [ ]:
train_nona_df = train_df.dropna(how="any")
train_nona_df.shape

In [ ]:
train_df.shape[0] - train_nona_df.shape[0]

In [ ]:
train_nona_df["Transported"].value_counts(dropna=False)

## Decision Tree

In [ ]:
y = train_df["Transported"]
# y.head()

In [ ]:
train_df.columns

In [ ]:
features = [
    "HomePlanet",
    "CryoSleep",
    "Cabin",
    "Destination",
    "Age",
    "VIP",
    "RoomService",
    "FoodCourt",
    "ShoppingMall",
    "Spa",
    "VRDeck",
]

X = train_df[features]
# X.head()

In [ ]:
dt_model = DecisionTreeClassifier(random_state=SEED)

dt_model.fit(X, y)

---